In [1]:
## 라이브러리 로드
import pandas as pd 
import numpy as np
from datetime import datetime

In [ ]:
df_OH = pd.read_csv('운영이력.csv') 
df_Bike=pd.read_csv('자전거스테이션.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 운영 이력 및 bike_Station 현황 분석

### 자전거 스테이션 이용 시간
- 데이터 컬럼의 'RTN_DATE'과 'LEAS_DATE' 차이를 통해 이용 시간을 추축해 'Bike_Time(s)' columns를 새로 만들어 준다.

In [ ]:
# 시간을 계산하기 위해 'LEAS_DATE','RTN_DATE'두개의 컬럼을 '년월일시분초' 형식으로 변경하여 준다 
df_OH['LEAS_DATE'] =  pd.to_datetime(df_OH['LEAS_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
df_OH['RTN_DATE'] =  pd.to_datetime(df_OH['RTN_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [ ]:
# 'RTN_DATE' - 'LEAS_DATE 를 통해 새로운 컬럼 초단위를 기준으로 하는 'Biek_Time(s)' 컬럼을 만들어 준다.
df_OH['Bike_Time(s)'] = df_OH['RTN_DATE'] - df_OH['LEAS_DATE'] # 이용 시간 Column 추가
df_OH['Bike_Time(s)'] = df_OH['Bike_Time(s)'].dt.total_seconds() #초단위로 나타내 준다.

In [ ]:
df_OH # Bike_Time(s)이 더해진 데이터 프레임을 만들어 준다.

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,Bike_Time(s)
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


운영 이력에 Bike_Time(s)이라는 칼럼을 통해 이용 시간을 분석할 수 있었다.

### 자전거 스테이션 이용 현황 파악
- 대여 및 반납 스테이션을 통해 기존 스테이션 비교해 보았습니다.
- 운영 이력에는 없지만 자전거 스테이션에 있는 데이터 파악
- 반납 스테이션 ==0 


In [ ]:
# 대여 및 반납 스테이션을 통해 기존 스테이션 비교해보기
LEAS_STATION = list(df_OH['LEAS_STATION'].sort_values(axis=0).unique()) # 데이터를 비교하기 위해 list로 변환
RTN_STATION = list(df_OH['RTN_STATION'].sort_values(axis=0).unique())
Bike_STATION = list(df_Bike['Station_ID'].unique()) 
print('LEAS_STATION 갯수:', len(LEAS_STATION))
print('rRTN_STATION 갯수:', len(RTN_STATION))
print('Bike_STATION 갯수:', len(Bike_STATION))

LEAS_STATION 갯수: 156
rRTN_STATION 갯수: 158
Bike_STATION 갯수: 164


In [ ]:
# 중복 데이터를 제거 해 주는 함수를 사용한다.
def remove_duplicated(a, b):                                                    
    c = set(a) | set(b)                                                         
    d = set(a) & set(b)                                                         
    e = c - d                                                                   
    return e     

In [ ]:
# 중복 데이터를 제거해 RTN_STATION,LEAS_STATION 차이를 구해본다.
RTN_STATION_LEAS_STATION = remove_duplicated(RTN_STATION, LEAS_STATION)
Bike_STATION_LEAS_STATION = remove_duplicated(Bike_STATION, LEAS_STATION)
print('RTN_STATION_LEAS_STATION:', RTN_STATION_LEAS_STATION) # 0,999 어떤 것들이 있는지 확인
print('2017-2019운영 이력에는 없는 자전거 스테이션',Bike_STATION_LEAS_STATION)

RTN_STATION_LEAS_STATION: {0, 999}
2017-2019운영 이력에는 없는 자전거 스테이션 {352, 353, 998, 241, 242, 151, 152, 153, 154, 155}


RTN_STATION과 LEAS_STATION의 차이를 분석해본 결과 RTN_STATION에서 (0,999) 두개의 스테이션이 더 있다는 것을 알 수 있었다. 그렇기에 RTN_STATION == 0, RTN_STATION == 99를 따로 분석해 보기로 했다. 

In [ ]:
# RTN_STATION ==999는 스테이션으로 반납하지 않은 경우를 나타낸다.
df_RTN_STATION_999 = df_OH[df_OH['RTN_STATION'] == 999]
df_RTN_STATION_999 

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,Bike_Time(s)
172317,16267550,2,2017-04-10 20:23:29,162,13,2017-04-11 05:11:12,999,99,NaN,3,188356,0.0,1A84400000F31C,5.0,31663.0
504175,16852299,2,2017-06-22 23:14:49,165,14,2017-06-22 23:34:03,999,99,NaN,3,194976,0.0,1A84400000F0C6,5.0,1154.0
628713,17067922,2,2017-07-24 00:36:24,171,1,2017-07-24 00:50:02,999,99,NaN,1,196777,0.0,1A84400000F332,5.0,818.0
1338134,18407241,2,2018-04-13 17:28:48,162,12,2018-04-14 00:45:44,999,99,NaN,3,206633,0.0,1A8440000006CE,5.0,26216.0
1494071,18697809,2,2018-05-29 18:31:28,161,2,2018-05-30 01:11:10,999,99,NaN,3,205400,0.0,1A84400000F53F,5.0,23982.0
1745962,19143907,2,2018-08-04 02:10:29,163,24,2018-08-04 02:16:47,999,99,NaN,6,199528,0.0,1A84400000F13A,5.0,378.0
1754909,19160499,2,2018-08-07 00:49:11,163,21,2018-08-07 00:59:30,999,99,NaN,6,199528,0.0,1A84400000F8A6,5.0,619.0
2238867,20065526,2,2019-03-05 23:27:56,172,18,2019-03-05 23:43:30,999,99,NaN,3,220675,0.0,1A84400000F81E,5.0,934.0
2259389,20105799,2,2019-03-17 00:15:37,168,20,2019-03-17 00:17:49,999,99,NaN,3,219868,0.0,1A84400000F51E,5.0,132.0
2488459,20530662,2,2019-05-31 23:58:31,172,7,2019-06-01 00:06:08,999,99,NaN,3,220675,0.0,1A84400000048C,5.0,457.0


In [ ]:
df_RTN_STATION_0 = df_OH[df_OH['RTN_STATION'] == 0]
df_RTN_STATION_0

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,Bike_Time(s)
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
14,15945559,2,2017-01-01 00:19:45,234,15,2017-01-01 00:33:12,0,0,NaN,12,180418,0.0,1A84400000F51A,1.0,807.0
18,15945563,2,2017-01-01 00:21:03,229,5,2017-01-01 01:21:03,0,0,NaN,99,0,3185087.0,1A84400000050D,1.0,3600.0
21,15945566,2,2017-01-01 00:24:58,306,7,2017-01-01 01:24:36,0,0,NaN,99,0,3185091.0,1A844000000B56,1.0,3578.0
26,15945573,2,2017-01-01 00:33:37,213,22,2017-01-01 00:49:06,0,0,NaN,12,180418,0.0,1A844000000768,1.0,929.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142655,21718573,2,2019-12-31 23:19:15,121,12,2020-01-01 00:15:09,0,0,NaN,1,236770,0.0,1A8440000006DD,1.0,3354.0
3142658,21718576,2,2019-12-31 23:19:54,214,3,2020-01-01 00:15:17,0,0,NaN,36,182974,0.0,1A84400000F532,1.0,3323.0
3142664,21718583,2,2019-12-31 23:25:53,343,15,2019-12-31 23:35:54,0,0,NaN,12,229770,0.0,1A84400000F823,1.0,601.0
3142679,21718602,2,2019-12-31 23:44:23,211,3,2020-01-01 00:40:49,0,0,NaN,12,196111,0.0,1A84400000FC8B,1.0,3386.0


RTN_STATION이 '0'인 데이터는 전체 데이터의 대략 8% 정도의 데이터가 추출이 되기 때문에 추후에 데이터를 함께 분석한다.

### 회원 및 비회원 구분
- 회원들과 비회원의 평균이용을 알 수 있다.(회원 : 22분 / 비회원 :36)


In [ ]:
df_member = df_OH[df_OH['MEMB_NO'] != 0] # 회원
df_nonmember = df_OH[df_OH['TEMP_MEMB_NO'] != 0] # 비회원

In [ ]:
## 회원/비회원의 Bike_Time(s)의 평균(이용 시간의 평균)을 구해준다
print('회원들의 평균 이용 시간(분) :',df_member['Bike_Time(s)'].mean()/60)
print('비회원들의 평균 이용 시간(분) :',df_nonmember['Bike_Time(s)'].mean()/60)

회원들의 평균 이용 시간(분) : 22.937086898305278
비회원들의 평균 이용 시간(분) : 36.18101054449756


In [ ]:
# 회원수 구하기
Members= set(df_member['MEMB_NO'])
Members_num = list(Members)
print('17-19년 자전거 회원 수 ;',len(Members_num ))

17-19년 자전거 회원 수 ; 39126


### 운영 이력 데이터 컬럼 변경
- 한글로 보기 싶게 컬럼 이름 변경
LEAS_NO    대여 번호  
LEAS_DATE    대여 시간  
LEAS_STATION    대여 스테이션 번호  
LEAS_DEF_NO    대여 거치대 번호  
RTN_DATE    반납 시간  
RTN_STATION    반납 스테이션 번호  
RTN_DEF_NO    반납 거치대 번호  
TRNV_QTY    추정 이동 거리  
MEMB_DIV    회원 구분(비회원은 99이며 나머지는 정회원)  
MEMB_NO    회원 번호  
TEMP_MEMB_NO    비회원 번호  
BIKE_TAG    자전거 번호  
Bike_Time(s) 이용 시간(s)

In [ ]:
del df_OH['LEAS_STAT'] # 설명 안된 데이터 프레임 제거

In [ ]:
df_OH

,LEAS_NO,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,Bike_Time(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


In [ ]:
df_OH.columns = ['대여 번호', '대여 시간', '대여 스테이션 번호', '대여 거치대 번호', '반납 시간','반납 스테이션 번호','반납 거치대 번호','추정 이동 거리', '회원 구분', '회원 번호', '비회원 번호', '자전거 번호', '관제 반납','이용 시간(s)']
# 컬럼 목록을 한글로 정리

In [ ]:
df_OH

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0
